<a href="https://colab.research.google.com/github/deepfake-detection-model-project/Deepfake-Detection-Project/blob/jiwon-dev/notebooks/02_modeling_exp/backbone_1st_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install timm # 현재 코랩에 이미 설치되어 있어서 주석처리 함

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from tqdm import tqdm
import pandas as pd
from google.colab import drive
import os
from collections import defaultdict
import gc

In [ ]:
# 1. 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
import sys
# src 폴더를 인식할 수 있도록 경로 추가 (현재 경로가 /content라면)
sys.path.append('/content/drive/MyDrive/HECTO')

from src.utils import set_seed
from src.dataset import DeepfakeDataset, seed_worker
from src.trainer import train_one_epoch, validate_video_level, save_checkpoint

In [ ]:
# 로컬에서 압축 해제 (네트워크 지연 없음)
!unzip -q /content/drive/MyDrive/HECTO/Dataset/images_data.zip -d /content/data_local

In [ ]:
# 2. 체크포인트 저장 경로 설정 (본인의 드라이브 경로에 맞게 수정 가능)
CHECKPOINT_DIR = '/content/drive/MyDrive/HECTO/checkpoints/01_backbone_selection'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# 3. 데이터셋 경로
CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/total_local_path.csv'

print(f"✅ 체크포인트 저장 경로: {CHECKPOINT_DIR}")

In [ ]:
# 시드 설정
EXPERIMENT_SEED = 42
set_seed(EXPERIMENT_SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 시드 {EXPERIMENT_SEED} 고정 완료 및 {device} 준비")

In [ ]:
g = torch.Generator()
g.manual_seed(EXPERIMENT_SEED)

# 기본 증강 설정
IMG_SIZE = 224

train_transform = transforms.Compose([
    # 1. 기하학적 변환 (256x256 원본 해상도에서 수행)
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),

    # 2. 화질 및 색상 변환
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
    ], p=0.3),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),

    # 3. 센터 크롭 (Resize 대신 사용)
    # 256x256 이미지의 중앙에서 224x224를 잘라내어 픽셀 왜곡을 방지합니다.
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),

    # 4. 텐서화 및 정규화
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

    # 5. 가려짐 대비 (ToTensor 이후 적용)
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)
])

val_transform = transforms.Compose([
    # 1. 센터 크롭
    # 학습 시와 동일하게 중앙 영역을 224x224로 잘라냅니다.
    transforms.CenterCrop((IMG_SIZE, IMG_SIZE)),

    # 2. 텐서화 및 정규화
    # 학습 시 사용한 파라미터와 완벽히 일치해야 합니다.
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_max = 0
        self.delta = delta

    def __call__(self, val_acc):
        score = val_acc
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

def load_checkpoint(model_name, model, optimizer):
    filename = os.path.join(CHECKPOINT_DIR, f"{model_name}_checkpoint.pth.tar")
    if os.path.isfile(filename):
        print(f"🔄 체크포인트 불러오는 중: {filename}")
        checkpoint = torch.load(filename)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        return start_epoch, best_acc
    else:
        print("🆕 이전 체크포인트 없음. 처음부터 학습을 시작합니다.")
        return 0, 0

In [ ]:
# 실험 대상 모델
model_names = ['resnet50', 'efficientnet_b0', 'vit_tiny_patch16_224', 'convnext_tiny']

# 하이퍼파라미터
MAX_EPOCHS = 20  # 얼리스탑이 있으므로 넉넉히 잡습니다.
BATCH_SIZE = 32
LR = 1e-4

# 결과를 저장할 리스트
results_summary = []

# 데이터 로더 준비
train_dataset = DeepfakeDataset(CSV_PATH, split='train', transform=train_transform)
val_dataset = DeepfakeDataset(CSV_PATH, split='val', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, worker_init_fn=seed_worker, generator=g)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)


In [ ]:
for m_name in model_names:
    print(f"\n" + "="*50)
    print(f"🚀 Experiment Start: {m_name}")
    print("="*50)

    set_seed(EXPERIMENT_SEED) # 시드 다시 한번 고정

    # 1. 모델 초기화 (Binary Classification을 위해 num_classes=1 설정)
    model = timm.create_model(m_name, pretrained=True, num_classes=1).to(device)

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LR)
    early_stopping = EarlyStopping(patience=5, verbose=True)

    # 체크포인트 불러오기 (중단된 경우 대비)
    start_epoch, best_acc = load_checkpoint(m_name, model, optimizer)

    for epoch in range(start_epoch, MAX_EPOCHS):
        print(f"\n[Epoch {epoch+1}/{MAX_EPOCHS}]")

        # 학습
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)

        # 검증 (영상 단위 평가)
        val_loss, video_acc = validate_video_level(model, val_loader, criterion, device)

        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val Loss:   {val_loss:.4f} | Video-level Val Acc: {video_acc:.4f}")

        # 베스트 모델 여부 확인
        is_best = video_acc > best_acc
        if is_best:
            best_acc = video_acc

        # 체크포인트 및 베스트 가중치 저장
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc': best_acc,
        }, is_best, m_name, CHECKPOINT_DIR)

        # 얼리스탑 확인
        early_stopping(video_acc)
        if early_stopping.early_stop:
            print(f"🛑 {m_name} Early Stopping!")
            break

    results_summary.append({'Model': m_name, 'Best Video Acc': best_acc})

    # 메모리 정리
    del model, optimizer
    torch.cuda.empty_cache()
    gc.collect()

# 최종 결과 출력
print("\n" + "🏆 Final Comparison Report " + "🏆")
df_res = pd.DataFrame(results_summary)
print(df_res.sort_values(by='Best Video Acc', ascending=False))